In [1]:
import pandas as pd
import os
import numpy as np
import numpy.ma as ma
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import hkvsobekpy as hkv
import sys
import netCDF4
import shutil
import subprocess
import fiona
import geopandas as gpd
import re
import math
import sys
#sys.path.insert(0, r'D:/Users/horn/Documents/3640.20/DamVanBrakel_vergelijking/Models')
#import resultreaderSubGrid as resultreader
import rasterio
import rioxarray
import xarray
#import gdal

from scipy import spatial
from shapely.ops import nearest_points
from scipy.spatial import cKDTree
from matplotlib import dates
from shutil import copyfile
from datetime import datetime as dt
from datetime import timedelta as td
from netCDF4 import Dataset
from netCDF4 import stringtochar
from netCDF4 import chartostring
from rasterio.transform import from_origin
#from pandas.plotting import register_matplotlib_converters
#register_matplotlib_converters()

%matplotlib inline

In [2]:
names = pd.DataFrame({'NaamHis':['QLAT.HIS','QLAT.OVERSTORT'],
     'NaamVarHis':['Lateral disch.(m3/s)','Defined Lateral Disc']}).set_index('NaamHis')

NaamHis='QLAT.HIS' #Naam van HIS bestand
#NaamVarHis='Lateral disch.(m3/s)' #Variabele naam in HIS bestand
NaamLateralsBC='FlowFM_lateral_sources.bc' #Naam voor BS bestand
directoryHis="../00_Data/sobekdata/SOBEK.lit/1/" #Locatie  his bestanden
directoryBC="../00_Data/sobekdata/Lateral_sources/" #Locatie  voor te maken bc bestand
starttijd = '2010-11-09 00:00:00'  #Starttijd voor in laterals.bc bestand
#dagen=13 #Aantal dagen voor tijdreeks in laterals.bc, let op dit zorgt voor een tijdreeks met de juiste lengte. Belangrijk dat dit overeenkomt met Sobekresultaat
tijdstapLaterals=60 #Tijdstap in seconden, moet zelfde zijn als in Sobek

In [3]:
naam_output=directoryBC+'/'+NaamLateralsBC
his_file = directoryHis+'/'+NaamHis
laterals = hkv.read_his.ReadMetadata(his_file)
sobek = laterals.DataFrame()
sobek_2 = pd.read_csv(r'D:\werkplan 2023\kennisagenda\23K01_dhydromodellering\01-importeer-neerslag-afvoer-uit-modellen\00_Data\output_afvoeren\timeseries\combinedoutput.csv', sep = ';', header=[0,1], index_col=0)
sobek=sobek_2

In [4]:
header ="""[General]
    fileVersion           = 1.01                
    fileType              = boundConds

"""

with open(naam_output, "w+") as f:
    f.write(header)
    f.close()

for i in names.index:
    NaamVarHis = names.loc[i,'NaamVarHis']

    sobek_Lat_def=sobek[NaamVarHis]
    afvoerreeks = sobek_Lat_def
    afvoerreeks.set_index(np.arange(0,len(afvoerreeks.index)*60,60), inplace=False)
    index = ((pd.to_datetime(afvoerreeks.index)- pd.to_datetime('2010-11-09 00:00:00', format='%Y-%m-%d'))/pd.Timedelta(minutes=1)).astype(int)
    afvoerreeks.set_index(index, inplace=True)
    afvoerreeks.dropna(axis=1,inplace=True,how='all')
    print(afvoerreeks)
    lateralen=[]
    for i in afvoerreeks.columns[0::]:
        naam = i
        tijdreeks = afvoerreeks.loc[:,i]
        template="""
    [forcing]
        name                  = {}           
        function              = timeseries          
        timeInterpolation     = linear              
        quantity              = time                
        unit                  = minutes since {}
        quantity              = lateral_discharge   
        unit                  = m\u00b3/s              
    {}
    """.format(naam,starttijd,tijdreeks.to_string())
        
        with open(naam_output, "a+") as f:
            f.write(template)
            f.close  

D:\temp\ipykernel_2320\3766794814.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  afvoerreeks.dropna(axis=1,inplace=True,how='all')


       AST-AST-CAM-15  AST-AST-HOO-8  AST-AST-IND-13  AST-AST-IND-24  \
60                0.0            0.0             0.0             0.0   
120               0.0            0.0             0.0             0.0   
180               0.0            0.0             0.0             0.0   
240               0.0            0.0             0.0             0.0   
300               0.0            0.0             0.0             0.0   
...               ...            ...             ...             ...   
18420             0.0            0.0             0.0             0.0   
18480             0.0            0.0             0.0             0.0   
18540             0.0            0.0             0.0             0.0   
18600             0.0            0.0             0.0             0.0   
18660             0.0            0.0             0.0             0.0   

       AST-AST-IND-25  AST-AST-IND-26  AST-AST-IND-27  AST-AST-KOM-1  \
60                0.0             0.0             0.0          

D:\temp\ipykernel_2320\3766794814.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  afvoerreeks.dropna(axis=1,inplace=True,how='all')


       1057660L  1055340L  1055935L  1056150L  1057380L  1057400L  1050020L  \
60     0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
120    0.000747  0.000547  0.001376  0.001635  0.000225  0.000178  0.000841   
180    0.000746  0.000546  0.001374  0.001632  0.000225  0.000178  0.000840   
240    0.000745  0.000545  0.001372  0.001630  0.000224  0.000178  0.000839   
300    0.000744  0.000544  0.001370  0.001627  0.000224  0.000177  0.000837   
...         ...       ...       ...       ...       ...       ...       ...   
18420  0.000579  0.000423  0.001066  0.001266  0.000174  0.000138  0.000651   
18480  0.000578  0.000423  0.001064  0.001265  0.000174  0.000138  0.000651   
18540  0.000577  0.000422  0.001063  0.001263  0.000174  0.000138  0.000650   
18600  0.000577  0.000422  0.001062  0.001262  0.000174  0.000137  0.000649   
18660  0.000577  0.000422  0.001062  0.001261  0.000174  0.000137  0.000649   

       1050120L  1050545L  1051250L  ...  1051710L 